Note: Multiplier has been taken as it is. For averages and ratios it doesn't affect athe analysis. To use absolute population values, use multiplier/100 instead

# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pathlib
import pickle
from functools import reduce

In [2]:
pd.set_option('display.max_columns', None)

# Importing datasets

In [3]:
# Checking paths
pathlib.Path("../../HCES 2023-24/Python implementation/Data extraction/Population based MPCE/MPCE 2023_24 2Apr25.dta").resolve().exists()

True

In [4]:
# Reading HH-wise MPCE
path_mpce = "../../HCES 2023-24/Python implementation/Data extraction/Population based MPCE/MPCE 2023_24 2Apr25.dta"
df_mpce = pd.read_stata(path_mpce)
df_mpce = df_mpce.set_index('hhid')
df_mpce = df_mpce[['hh_size', 'sector', 'mpce', 'decile']]
display(df_mpce.head())

## Level 01 file
path_lvl01 = "../../HCES 2023-24/Dta raw files//level_01.dta"
df_lvl01 = pd.read_stata(path_lvl01)
df_lvl01 = df_lvl01.set_index('hhid')
df_lvl01 = df_lvl01[['state']]
display(df_lvl01.head())

## Level 03 file
path_lvl03 = "../../HCES 2023-24/Dta raw files//level_03.dta"
df_lvl03 = pd.read_stata(path_lvl03)
df_lvl03 = df_lvl03.set_index('hhid')
df_lvl03 = df_lvl03[['source_cooking', 'multiplier']]
display(df_lvl03.head())

## Level 07 file
path_lvl07 = "../../HCES 2023-24/Dta raw files/level_07.dta"
df_lvl07 = pd.read_stata(path_lvl07)
df_lvl07 = df_lvl07.set_index('hhid')
df_lvl07 = df_lvl07[['received_subsidy_lpg', 'num_subsidized_lpg']]
display(df_lvl07.head())

## Level 08 file
path_lvl08 = "../../HCES 2023-24/Dta raw files/level_08.dta"
df_lvl08 = pd.read_stata(path_lvl08)
df_lvl08 = df_lvl08.loc[df_lvl08['item_code'].isin([331,338])]

,hh_size,sector,mpce,decile
hhid,,,,
22300101,4.0,2,9407.24,9
22300201,4.0,2,25398.56,10
22300202,2.0,2,11762.50,10
22300203,2.0,2,11579.24,10
22300204,2.0,2,15350.10,10


,state
hhid,
46667201,1
46667301,1
46667302,1
46667303,1
46667304,1


,source_cooking,multiplier
hhid,,
22300101,2,57436
22300201,2,27497
22300202,2,27497
22300203,2,27497
22300204,2,27497


,received_subsidy_lpg,num_subsidized_lpg
hhid,,
22300101,1,2.0
22300201,1,2.0
22300202,1,1.0
22300203,2,NaN
22300204,1,1.0


# Data wrangling

In [5]:
df_lvl08.head()

,index,hhid,questionnaire_num,level,item_code,cons_home_qty,cons_home_value,cons_total_qty,cons_total_value,source,multiplier
5,5,22300101,C,8,338,NaN,NaN,14.2,860.0,1.0,57436
9,9,22300201,C,8,338,NaN,NaN,14.2,860.0,1.0,27497
16,16,22300202,C,8,338,NaN,NaN,9.5,640.0,1.0,27497
23,23,22300203,C,8,338,NaN,NaN,9.0,720.0,1.0,27497
26,26,22300204,C,8,338,NaN,NaN,9.4,570.0,1.0,27497


In [6]:
# Reshaping level 08 file for firewood and LPG

## Pivoting
df_lvl08 = df_lvl08.pivot_table(index = 'hhid', columns = 'item_code', values = ['cons_total_qty','cons_total_value', 'cons_home_qty', 'cons_home_value' ])

## Converting columns to single level index
df_lvl08.columns = ["_".join(map(str, x)) for x in df_lvl08.columns]

## Changing column names
df_lvl08 = df_lvl08.rename(columns = {'cons_total_qty_331': 'firewood_qty', 'cons_total_qty_338': 'lpg_qty',
                               'cons_total_value_331': 'firewood_value', 'cons_total_value_338': 'lpg_value',
                               'cons_home_qty_331': 'firewood_home_qty', 'cons_home_value_331': 'firewood_home_value',
                               'cons_home_qty_338': 'lpg_home_qty', 'cons_home_value_338': 'lpg_home_value'})
df_lvl08.head()

# Adding market quantity and value
df_lvl08 = df_lvl08.fillna(0)
df_lvl08[['firewood_mkt_qty', 'firewood_mkt_value']] = df_lvl08[['firewood_qty', 'firewood_value' ]].values - df_lvl08[['firewood_home_qty', 'firewood_home_value' ]].values
df_lvl08[['lpg_mkt_qty', 'lpg_mkt_value']] = df_lvl08[['lpg_qty', 'lpg_value' ]].values - df_lvl08[['lpg_home_qty', 'lpg_home_value' ]].values

In [7]:
df_lvl08.head()

,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,
22300101,0.0,0.0,0.0,0.0,0.0,14.2,0.0,860.0,0.0,0.0,14.2,860.0
22300201,0.0,0.0,0.0,0.0,0.0,14.2,0.0,860.0,0.0,0.0,14.2,860.0
22300202,0.0,0.0,0.0,0.0,0.0,9.5,0.0,640.0,0.0,0.0,9.5,640.0
22300203,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
22300204,0.0,0.0,0.0,0.0,0.0,9.4,0.0,570.0,0.0,0.0,9.4,570.0


In [8]:
# Merging all files

## Checking shapes
print(f"Shape of MPCE file: {df_mpce.shape}")
print(f"Shape of level 01 file: {df_lvl01.shape}")
print(f"Shape of level 03 file: {df_lvl03.shape}")
print(f"Shape of level 07 file: {df_lvl07.shape}")
print(f"Shape of level 08 file: {df_lvl08.shape}")

## Defining merge function
def merge_df (left_df, right_df):
    df = pd.merge(left_df, right_df, how = 'outer', left_index=True, right_index=True, indicator=True)
    print(f"\nSummary from merging dataframes")
    display(df['_merge'].value_counts())
    df = df.drop(labels = '_merge', axis = 1)
    return df.copy()

## Merging
df = reduce(merge_df, [df_mpce, df_lvl01, df_lvl03, df_lvl07, df_lvl08])

Shape of MPCE file: (261953, 4)
Shape of level 01 file: (261953, 1)
Shape of level 03 file: (261953, 2)
Shape of level 07 file: (261953, 2)
Shape of level 08 file: (251119, 12)

Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          251119
left_only      10834
right_only         0
Name: count, dtype: int64

In [9]:
# Renaming column values
df['sector'] = df['sector'].replace({1: 'rural', 2: 'urban'})
df['state'] = df['state'].replace({1: 'Jammu & Kashmir', 2: 'Himachal Pradesh', 3: 'Punjab', 4: 'Chandigarh(U.T.)', 5: 'Uttrakhand', 6: 'Haryana', 7: 'Delhi', 8: 'Rajasthan', 9: 'Uttar Pradesh', 10: 'Bihar', 11: 'Sikkim', 12: 'Arunachal Pradesh', 13: 'Nagaland', 14: 'Manipur', 15: 'Mizoram', 16: 'Tripura', 17: 'Meghalaya', 18: 'Assam', 19: 'West Bengal', 20: 'Jharkhand', 21: 'Odisha', 22: 'Chattisgarh', 23: 'Madhya Pradesh', 24: 'Gujarat', 25: 'Dadra & Nagar Haveli', 27: 'Maharashtra', 28: 'Andhra Pradesh', 29: 'Karnataka', 30: 'Goa', 31: 'Lakshadweep (U.T.)', 32: 'Kerala', 33: 'Tamilnadu', 34: 'Puducherry (U.T.)', 35: 'A and N Islands (U.T.)', 36: 'Telangana', 37: 'Ladakh (U.T.)'})
df['source_cooking'] = df['source_cooking'].replace({1: "firewood and chips", 2: "LPG", 3: "other natural gas", 4: "dung cake", 5: "kerosene", 6: "coke, coal", 7: "gobar gas", 8: "other biogas", 9: "others", 10: "charcoal", 11: "electricity", 12: "no cooking arrangement"})
df['received_subsidy_lpg'] = df['received_subsidy_lpg'].replace({1: 'yes', 2: 'no'})

# Analysis

## Primary cooking source

In [10]:
# Across India

## Calculating
trial = df.groupby(['source_cooking'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,source_cooking,popn_size,sample_size
0,LPG,17526505096,3371620
1,charcoal,2625950,2340
2,"coke, coal",88458521,19560
3,dung cake,435423977,63580
4,electricity,29968358,6880
5,firewood and chips,9507038104,1556040
6,gobar gas,14940537,2140
7,kerosene,10668411,2060
8,no cooking arrangement,877342119,103600
9,other biogas,915657,160


In [36]:
# Primary cooking source (Rural vs Urban)

## Calculating
trial = df.groupby(['sector','source_cooking']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
trial = trial.reset_index()

## Pivoting
trial = trial.pivot_table(values = 'popn_size', index = ['sector'], columns = 'source_cooking')
display(trial)

source_cooking,LPG,charcoal,"coke, coal",dung cake,electricity,firewood and chips,gobar gas,kerosene,no cooking arrangement,other biogas,other natural gas,others
sector,,,,,,,,,,,,
rural,9.332794e+09,808287.0,51522295.0,419645451.0,13837302.0,9.044997e+09,14515830.0,1049584.0,210825486.0,698248.0,20541726.0,267149057.0
urban,8.193711e+09,1817663.0,36936226.0,15778526.0,16131056.0,4.620413e+08,424707.0,9618827.0,666516633.0,217409.0,356119213.0,29411059.0


In [37]:
# Primary cooking source (Rural vs Urban by decile)

## Calculating
trial = df.groupby(['sector','decile','source_cooking'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)


## Pivoting
trial = trial.pivot_table (values = 'popn_size', index = ['sector', 'source_cooking'], columns = 'decile', observed = False)
display(trial)

decile                                   1             2             3   \
sector source_cooking                                                     
rural  LPG                     2.189766e+08  3.828172e+08  5.028727e+08   
       charcoal                         NaN           NaN           NaN   
       coke, coal              9.042603e+06  9.174822e+06  7.087750e+06   
       dung cake               8.987286e+07  6.345189e+07  4.903431e+07   
       electricity             2.098300e+04  3.245500e+05  5.802490e+05   
       firewood and chips      1.087077e+09  1.083879e+09  1.046656e+09   
       gobar gas               1.345600e+05  4.783100e+05  4.993600e+05   
       kerosene                         NaN           NaN           NaN   
       no cooking arrangement  8.646974e+07  8.209514e+06  4.601355e+06   
       other biogas                     NaN           NaN           NaN   
       other natural gas                NaN  1.823730e+05  2.612990e+05   
       others                  1.921939e+07  2.693879e+07  2.714670e+07   
urban  LPG                     4.741037e+08  6.249021e+08  7.043255e+08   
       charcoal                8.781720e+05  7.012000e+04  2.075550e+05   
       coke, coal              2.005107e+07  7.351638e+06  3.728170e+06   
       dung cake               6.080005e+06  2.648526e+06  2.594818e+06   
       electricity             7.687430e+05  1.706690e+05  8.541250e+05   
       firewood and chips      1.395108e+08  8.841300e+07  6.324348e+07   
       gobar gas                        NaN           NaN           NaN   
       kerosene                8.348250e+05  6.795530e+05  1.306237e+06   
       no cooking arrangement  4.317840e+07  1.117255e+07  1.208167e+07   
       other biogas                     NaN           NaN           NaN   
       other natural gas       1.652607e+06  4.494963e+06  7.791580e+06   
       others                  4.910799e+06  2.106623e+06  1.347099e+06   

decile                                   4             5            6   \
sector source_cooking                                                    
rural  LPG                     6.219698e+08  7.151531e+08  882606503.0   
       charcoal                         NaN           NaN     130249.0   
       coke, coal              7.153578e+06  4.913307e+06    4261101.0   
       dung cake               4.540076e+07  3.698500e+07   38407689.0   
       electricity             5.042200e+05  3.188690e+05     546929.0   
       firewood and chips      1.031581e+09  1.005474e+09  937172769.0   
       gobar gas               8.361800e+05  1.491959e+06    2438026.0   
       kerosene                         NaN           NaN          NaN   
       no cooking arrangement  5.117404e+06  3.697131e+06    4227705.0   
       other biogas                     NaN           NaN          NaN   
       other natural gas       1.839740e+05  1.112773e+06     774016.0   
       others                  2.906683e+07  3.493363e+07   31711128.0   
urban  LPG                     7.563867e+08  8.031994e+08  852877280.0   
       charcoal                8.208100e+04  2.228490e+05      83894.0   
       coke, coal              3.026594e+06  1.209048e+06     487747.0   
       dung cake               1.604730e+06  1.085603e+06     746360.0   
       electricity             7.178720e+05  8.214410e+05    1587000.0   
       firewood and chips      4.898259e+07  3.500453e+07   24677113.0   
       gobar gas                        NaN  3.002510e+05     118280.0   
       kerosene                1.119441e+06  1.795848e+06    1360376.0   
       no cooking arrangement  2.371192e+07  2.291889e+07   30480321.0   
       other biogas                     NaN           NaN          NaN   
       other natural gas       1.537602e+07  2.043582e+07   24110077.0   
       others                  2.191255e+06  1.873442e+06    3065227.0   

decile                                   7             8             9   \
sector source_cooking                            

## LPG subsidy - Last three months

In [39]:
# Across India
trial = df.groupby(['received_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,received_subsidy_lpg,popn_size,sample_size
0,no,20474106814,2009073
1,yes,8693000971,872410


In [41]:
# LPG subsidy (Rural vs Urban)
trial = df.groupby(['sector','received_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)

# Pivoting 
trial = trial.pivot_table(values = 'popn_size', index = 'received_subsidy_lpg', columns = 'sector')
display(trial)

sector,rural,urban
received_subsidy_lpg,,
no,1.339606e+10,7.078048e+09
yes,5.982325e+09,2.710676e+09


In [43]:
# LPG subsidy (Rural vs Urban by decile)
trial = df.groupby(['sector','decile','received_subsidy_lpg'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'wt_sum': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)

# Pivoting
trial = trial.pivot_table(values = 'wt_sum', index = ['sector','received_subsidy_lpg'], columns = 'decile', observed = False)
display(trial)

decile                                 1             2             3   \
sector received_subsidy_lpg                                             
rural  no                    1.313577e+09  1.238561e+09  1.215671e+09   
       yes                   1.972371e+08  3.368950e+08  4.230677e+08   
urban  no                    4.593144e+08  4.781740e+08  5.129571e+08   
       yes                   2.326547e+08  2.638358e+08  2.845231e+08   

decile                                 4             5             6   \
sector received_subsidy_lpg                                             
rural  no                    1.237141e+09  1.240051e+09  1.262182e+09   
       yes                   5.046728e+08  5.640280e+08  6.400944e+08   
urban  no                    5.707477e+08  6.160736e+08  6.469676e+08   
       yes                   2.824515e+08  2.727936e+08  2.926261e+08   

decile                                 7             8             9   \
sector received_subsidy_lpg                                             
rural  no                    1.311752e+09  1.367503e+09  1.436729e+09   
       yes                   7.085627e+08  7.717991e+08  8.649423e+08   
urban  no                    7.012140e+08  7.709213e+08  9.077767e+08   
       yes                   2.938841e+08  2.926048e+08  2.742496e+08   

decile                                 10  
sector received_subsidy_lpg                
rural  no                    1.772891e+09  
       yes                   9.710259e+08  
urban  no                    1.413901e+09  
       yes                   2.210527e+08

# LPG susbsidised cylinder - Last 3 months

In [45]:
# Across India

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby('num_subsidized_lpg', as_index = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

,num_subsidized_lpg,popn_size,sample_size
0,1.0,5056143332,490358
1,2.0,2612575002,267190
2,3.0,1024282637,114862


In [47]:
# Rural vs Urban

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby(['sector','num_subsidized_lpg']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
display(trial)

popn_size  sample_size
sector num_subsidized_lpg                         
rural  1.0                 3776872469       286400
       2.0                 1614754868       123300
       3.0                  590697723        46280
urban  1.0                 1279270863       159380
       2.0                  997820134       119600
       3.0                  433584914        58140

In [49]:
# Rural vs Urban across deciles

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

## Calculation
trial = trial.groupby(['sector', 'decile', 'num_subsidized_lpg'], observed = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.size}
), include_groups = False)
trial = trial.reset_index()

# Pivotting
trial = trial.pivot_table(values = 'popn_size', index = ['sector', 'num_subsidized_lpg'], columns = 'decile',observed = False)
display(trial)

decile                              1            2            3            4   \
sector num_subsidized_lpg                                                       
rural  1.0                 140107176.0  230722555.0  274815471.0  319540048.0   
       2.0                  37219876.0   71990211.0  102651297.0  128427957.0   
       3.0                  19910069.0   34182231.0   45600968.0   56704778.0   
urban  1.0                 108230394.0  114501517.0  128067840.0  131044393.0   
       2.0                  76354036.0  100424229.0  105577908.0  104207601.0   
       3.0                  48070269.0   48910005.0   50877390.0   47199500.0   

decile                              5            6            7            8   \
sector num_subsidized_lpg                                                       
rural  1.0                 362214360.0  400636155.0  445181255.0  480067795.0   
       2.0                 143782710.0  178556303.0  195806045.0  220797532.0   
       3.0                  58030934.0   60901972.0   67575374.0   70933780.0   
urban  1.0                 129655352.0  134644461.0  145044423.0  143178678.0   
       2.0                 101535385.0  114681964.0  105762012.0  112875196.0   
       3.0                  41602846.0   43299644.0   43077696.0   36550929.0   

decile                              9            10  
sector num_subsidized_lpg                            
rural  1.0                 535293192.0  588294462.0  
       2.0                 252183285.0  283339652.0  
       3.0                  77465854.0   99391763.0  
urban  1.0                 138563443.0  106340362.0  
       2.0                  94695398.0   81706405.0  
       3.0                  40990732.0   33005903.0

# Average quantity and value of LPG and firewood consumed

* Values are reported for past 30 days

In [106]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum(),
    'lpg_value':(trial['lpg_value'] * trial['multiplier']).sum(),
    'firewood_qty': (trial['firewood_qty'] * trial['multiplier']).sum(),
    'firewood_value':(trial['firewood_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

lpg_qty             1.598124
lpg_value         106.070636
firewood_qty        9.937351
firewood_value     46.950929
dtype: float64

In [51]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_lpg_qty,wt_lpg_value,wt_firewod_qty,wt_firewood_value
sector,,,,
rural,1.270472,84.779279,13.300155,62.60823
urban,2.369823,156.216889,2.017142,10.07423


In [52]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0).round(2)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewod_qty  wt_firewood_value
sector decile                                                             
rural  1             0.39         26.79           15.03              66.63
       2             0.66         44.75           15.22              69.26
       3             0.83         55.93           14.96              69.12
       4             0.99         66.54           14.43              68.61
       5             1.10         73.44           13.90              66.80
       6             1.27         84.82           13.48              64.89
       7             1.46         97.54           12.59              60.64
       8             1.66        110.09           12.01              58.06
       9             1.92        127.53           11.07              53.08
       10            2.41        160.36           10.31              48.98
urban  1             1.58        104.62            4.88              24.00
       2             1.99        131.35            3.34              17.37
       3             2.18        143.11            2.62              13.23
       4             2.27        149.70            2.12              10.36
       5             2.38        156.70            1.91               9.31
       6             2.49        163.90            1.44               7.25
       7             2.61        171.94            1.35               6.70
       8             2.67        175.65            1.12               5.35
       9             2.79        183.74            0.79               3.98
       10            2.74        181.46            0.61               3.20

# Average quantity and value of firewood consumed from market

In [110]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'firewood_mkt_qty': (trial['firewood_mkt_qty'] * trial['multiplier']).sum(),
    'firewood_mkt_value':(trial['firewood_mkt_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

firewood_mkt_qty       6.965658
firewood_mkt_value    33.798832
dtype: float64

In [125]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_firewood_mkt_qty,wt_firewood_mkt_value
sector,,
rural,9.254475,44.566351
urban,1.574948,8.438745


In [127]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile']).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile']).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector decile                                            
rural  1                 12.304579              54.413062
       2                 11.530213              53.543743
       3                 10.885525              51.120064
       4                 10.227797              49.686170
       5                  9.593851              47.209538
       6                  8.857644              43.942270
       7                  8.304052              41.296977
       8                  7.827312              39.188145
       9                  7.015588              35.049399
       10                 5.997914              30.213107
urban  1                  4.331902              21.566056
       2                  2.848006              15.340643
       3                  2.072258              11.267240
       4                  1.646409               8.712177
       5                  1.420490               7.586944
       6                  0.987593               5.683504
       7                  0.869814               5.074928
       8                  0.696843               3.911090
       9                  0.510051               2.984900
       10                 0.365950               2.259185

In [27]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
rural  A and N Islands (U.T.)             1.297966               6.060627
       Andhra Pradesh                     6.337395              26.043898
       Arunachal Pradesh                 16.854789             111.634300
       Assam                             12.851265              65.990285
       Bihar                              5.368933              38.197365
...                                            ...                    ...
urban  Telangana                          0.123115               0.857575
       Tripura                            8.572852              46.564274
       Uttar Pradesh                      1.002565               6.393113
       Uttrakhand                         1.169996               5.838457
       West Bengal                        2.846219              15.128429

[72 rows x 2 columns]

In [28]:
trial[trial.index.get_level_values(0) == 'urban']

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
urban  A and N Islands (U.T.)             0.214033               0.853284
       Andhra Pradesh                     0.755151               3.791601
       Arunachal Pradesh                  9.150919              60.658396
       Assam                              1.506175               7.378310
       Bihar                              1.645396              11.821243
       Chandigarh(U.T.)                   0.026843               0.214747
       Chattisgarh                        2.280275              12.776171
       Dadra & Nagar Haveli               2.314554              11.806258
       Delhi                              0.021515               0.193664
       Goa                                0.642300               2.568494
       Gujarat                            0.749301               3.966844
       Haryana                            0.596889               4.142647
       Himachal Pradesh                   2.037785               6.597161
       Jammu & Kashmir                    1.329779               5.186638
       Jharkhand                          2.249808               9.812565
       Karnataka                          2.102091              10.894142
       Kerala                             6.372496              31.105580
       Ladakh (U.T.)                      0.116185               1.036975
       Lakshadweep (U.T.)                 3.247482              18.367614
       Madhya Pradesh                     1.075819               4.794654
       Maharashtra                        0.502864               2.584978
       Manipur                            0.489822               4.664711
       Meghalaya                          2.963309              15.896506
       Mizoram                            0.000000               0.000000
       Nagaland                           9.326515              37.503127
       Odisha                             8.872845              42.164875
       Puducherry (U.T.)                  0.118627               1.372400
       Punjab                             0.526855               4.339808
       Rajasthan                          2.707494              14.344861
       Sikkim                             0.284645               0.755628
       Tamilnadu                          0.685830               5.276920
       Telangana                          0.123115               0.857575
       Tripura                            8.572852              46.564274
       Uttar Pradesh                      1.002565               6.393113
       Uttrakhand                         1.169996               5.838457
       West Bengal                        2.846219              15.128429

# Average quantity and value of lpg consumed

In [32]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty'] * table['multiplier']).sum(),
    'wt_lpg_value':(table['lpg_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_lpg_qty  wt_lpg_value
sector state                                           
rural  A and N Islands (U.T.)    2.935685    193.343921
       Andhra Pradesh            1.881545    127.873048
       Arunachal Pradesh         1.257256     88.357207
       Assam                     0.990500     66.470068
       Bihar                     1.095967     77.998667
...                                   ...           ...
urban  Telangana                 2.047571    138.812165
       Tripura                   2.366207    179.032254
       Uttar Pradesh             2.279728    147.500488
       Uttrakhand                2.800773    178.980782
       West Bengal               2.789188    180.640258

[72 rows x 2 columns]

In [34]:
trial[trial.index.get_level_values(0) == 'urban']

wt_lpg_qty  wt_lpg_value
sector state                                           
urban  A and N Islands (U.T.)    3.446384    235.749064
       Andhra Pradesh            2.180879    147.770615
       Arunachal Pradesh         2.411137    173.941360
       Assam                     2.604416    177.177031
       Bihar                     2.236689    160.099878
       Chandigarh(U.T.)          2.925051    201.796932
       Chattisgarh               2.318764    158.815367
       Dadra & Nagar Haveli      2.523876    163.694446
       Delhi                     2.249330    145.565912
       Goa                       3.187415    198.297461
       Gujarat                   1.569232    103.851594
       Haryana                   2.855434    180.579407
       Himachal Pradesh          3.018666    214.351777
       Jammu & Kashmir           2.353751    166.895077
       Jharkhand                 2.084631    144.911570
       Karnataka                 2.400978    156.425165
       Kerala                    2.439940    159.874414
       Ladakh (U.T.)             2.362640    180.447101
       Lakshadweep (U.T.)        1.584744    157.576844
       Madhya Pradesh            2.129490    142.023456
       Maharashtra               2.464070    158.547105
       Manipur                   1.996565    158.610834
       Meghalaya                 2.286423    170.392722
       Mizoram                   3.528328    239.288750
       Nagaland                  2.178764    173.176309
       Odisha                    2.243735    148.004000
       Puducherry (U.T.)         2.929160    201.693910
       Punjab                    3.107593    202.303450
       Rajasthan                 2.281580    146.731027
       Sikkim                    2.743746    248.896237
       Tamilnadu                 2.634196    178.476598
       Telangana                 2.047571    138.812165
       Tripura                   2.366207    179.032254
       Uttar Pradesh             2.279728    147.500488
       Uttrakhand                2.800773    178.980782
       West Bengal               2.789188    180.640258

# Household average annual cylinder consumption

In [19]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum()/trial['multiplier'].sum(),
})

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

lpg_qty    5.64
dtype: float64

In [21]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False)

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

,wt_lpg_qty
sector,
rural,4.74
urban,7.42


In [22]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False).round(2)

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

wt_lpg_qty
sector decile            
rural  1             1.88
       2             3.04
       3             3.67
       4             4.12
       5             4.39
       6             4.83
       7             5.23
       8             5.59
       9             6.04
       10            6.35
urban  1             7.01
       2             8.22
       3             8.37
       4             8.17
       5             8.21
       6             8.13
       7             8.03
       8             7.70
       9             7.24
       10            5.15